Попробуем решить следующую задачу: нам необходимо выгрузить сайт NPlus1.ru и положить статьи в базу данных. Отдельно должны лежать идентификаторы загруженных текстов, словари начальных форм и токенов, разбиение текста на предложения. Для решения задачи будем использовать базу данных MongoDB.

Импортируем все необходисые библиотеки. Обратите особое внимание на библиотеку для работы с Mongo - <a href="http://api.mongodb.com/python/current/api/index.html">pymongo</a> (еще одно описание <a href="https://pymongo.readthedocs.io/en/stable/">здесь</a>).

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import datetime
from tqdm import tqdm
import pymorphy2

import pymongo
from bson import ObjectId
import json

delcom=re.compile("<!--.+-->", re.S)


Чтобы не ставить себе базу данных, можно использовать бесплатное облако <a href ="https://cloud.mongodb.com/">Mongo.Atlas</a>. Для этого необходимо зарегистрироваться, завести пользователя, создать проект, дать пользователю права на проект. Для пользователя получить строку для соединения с базой. 

Но в данном случае мы будем пользоваться MongoDB, запущенное на локальном компьютере при помощи Docker.

Для работы с БД используем класс MongoClient, в который передаем строку соединения.

In [2]:
# Version after 3.6
client = pymongo.MongoClient('localhost', 27017)
# client = pymongo.MongoClient('localhost', 8081)
#client = pymongo.MongoClient()

Аналогом реляционных записей являются документы. Аналогом таблиц - коллекции и подколлекции. Сам документ может включать в себя другие документы. Документ записывается в формате JSON, но для Питона можно считать, что это словари.<br>
При помощи клиента обращаемся к базе `concordance`. Если эта база не существовала, она создастся при первой записи данных. Аналогично всё происходит с коллекциями.<br>

В коллекции `nplus1texts` будем хранить адерса и идентификаторы текстов, в коллекции `dictionary` - словарь токенов, в `lemmas` - словарь лемм, наконец в `sentences` - разделение предложений на слова.

In [3]:
# Обращение к базе при помощи оператора квадратные скобки.
db = client['concordance']
# Обращение к базе при помощи оператора точка.
#db = client.concordance

Посмотрим что содержит в себе база.

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'concordance')

In [5]:
# Обращение к коллекции внутри базы при помощи оператора точка (но можно и при помощи квадратных скобок).
text_collection = db.nplus1texts
dictionary = db.dictionary
lemmas = db.lemmas
dbsents = db.sentences

In [6]:
# Добавляем индекс в коллекцию lemmas по полям iniForm и POS, оба по возрастанию.
# Заодно запрещаем добавлять повторяющиеся сочетания.
lemmas.create_index([('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)
dictionary.create_index([('token', pymongo.ASCENDING), ('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)

'token_1_iniForm_1_POS_1'

In [7]:
# Для пробы чуть позже удалим индекс и посмотрим на изменение скорости работы.
lemmas.drop_index('iniForm_1_POS_1')

Напишем функцию для выгрузки одной статьи с сайта (это мы уже умеем).

In [16]:
from dataclasses import dataclass
from lxml import html
import requests
from pprint import pprint

In [9]:
@dataclass
class LentaArticle:
    title: str
    text: str
    description: str
    time: str = "00:00"
    author: str = "No author"
        
def get_lenta_article(url: str) -> LentaArticle:
    page = requests.get(url)
    tree = html.fromstring(page.text)
    ttl = tree.xpath(".//h1")[0].text_content()
    dscrptn = tree.xpath(".//meta[@property='og:description']")[0].get("content")

    txt = '\n'.join([p.text_content() for p in 
             tree.xpath(".//div[contains(@class, '_news')]//p[contains(@class, 'topic-body__content-text')]")]
                    )
    
    article = LentaArticle(ttl, txt, dscrptn)
    article.time = tree.xpath(".//a[contains(@class, 'topic-header__time')]")[0].text_content().strip()
    article.author = tree.xpath(".//div[contains(@class, 'topic-authors')]")[0].text_content().strip()
    return article

def get_lenta_day(url):
    page = requests.get(url)
    tree = html.fromstring(page.text)
    urls = ["https://lenta.ru" + t.get("href") for t in tree.xpath(".//a[contains(@class, '_archive')]")]
    arts = [get_lenta_article(url) for url in urls]
    return arts
    
get_lenta_article('https://lenta.ru/news/2021/02/27/apple_effect/')

LentaArticle(title='Обнаружен неожиданный эффект от употребления яблок', text='Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.\nОпыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.\nПо словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффект, что и физическая активность, которая также стимулирует нейрогенез.\nРанее ученые из Технологического университета австрийского Граца выяснили, что большинство людей неправильно едят яблоки. Исследователи утверждают, что до 90 процентов полезных веществ сосредоточены в сердцевине этого фрукта, и п

In [21]:
class NPlus1Article:
    def __init__(self):
        self.time=""
        self.date=""
        self.rubr=""
        self.diff=""
        self.author=""
        self.head=""
        self.text=""
        
    def __repr__(self):
        s = f"{self.head}\n{self.date} : {self.time} : {self.rubr} : {self.diff}\n"
        s += f"{self.author}\n{self.text}"
        return s

def getArticleTextNPlus1(adr):
    r = requests.get(adr)
    art=NPlus1Article()
    tree = html.fromstring(r.text)

    art.head = tree.xpath(".//h1")[0].text_content().strip()
    art.author = tree.xpath(".//span[contains(@class, 'whitespace-nowrap')]")[0].text_content().strip()
    art.time = tree.xpath(".//span[contains(@class, 'duration-75')]")[0].text_content()
    art.date = tree.xpath(".//span[contains(@class, 'duration-75')]")[1].text_content()
    art.diff = tree.xpath(".//span[contains(@class, 'duration-75')]")[2].text_content()
    art.rubr = tree.xpath(".//span[contains(@class, 'duration-75')]")[3].text_content()
    art.text = '\n'.join([p.text_content() for p in 
                 tree.xpath(".//div[contains(@class, 'mb-14')]//p[contains(@class, 'mb-6')]")]
                        )
    return art

def getNPlus1Day(adr):
    r = requests.get(adr)
    a = re.findall("JSON.parse\('(.+?)'", r.text, re.DOTALL)[0].\
           replace('\\\\', '\\').\
           encode().\
           decode('unicode_escape').\
           replace('\\/', '/')
    b = json.loads(a)
    urls = [c['url'] for c in b['data']]
    results = []
    for url in urls:
        try:
            art = getArticleTextNPlus1(url)
            results.append(art)
        except:
            print(f"Exception: {url}")
    return results

In [11]:
getNPlus1Day("https://nplus1.ru/news/2022/11/24")

[
             Физики увидели запутанность продуктов распада куперовской пары
           
 24.11.22 : 15:45 : Физика : 7.8
 
                                       
                   
                   Марат Хамадеев
                 
 Физики из Италии и Швейцарии экспериментально доказали наличие антикорреляции у электронов, родившихся в результате распада куперовской пары в сверхпроводнике. Для этого они измеряли соответствующие токи по обе стороны от сверхпроводника, фильтруя их по спинам. Исследование опубликовано в Nature.
 Совокупность фермионов в общем случае сохраняет статистические свойства отдельных частиц. Но все меняется, если фермионов четное число и между ними возникает достаточно интенсивная связь, которая может поддерживать квантовую корреляцию. В этом случае фермионные пары можно объединить и рассматривать их в качестве одной частицы, подчиняющейся статистике Бозе — Эйнштейна.
 Наиболее понятным примером этого механизма можно назвать атом водорода. И электрон, и прот

Для того, чтобы найти документ, необходимо использовать функцию `collection.find()`. Без параметров эта функция выдаст все документы.

In [17]:
for t in text_collection.find():
    pprint(t)

{'_id': ObjectId('637f6f0d893f7efe906949d9'),
 'art_info': {'art_date': '20.01.20',
              'art_time': '14:18',
              'author': '\n'
                        '                                      \n'
                        '                  \n'
                        '                  Александр Войтюк\n'
                        '                ',
              'difficulty': '2.3'},
 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к '
             'Красной планете этим летом, успешно прошел заключительные '
             'термовакуумные испытания, имитировавшие\xa0условия марсианской '
             'среды, в которых предстоит работать роверу,\xa0сообщается на '
             'сайте ESA.\n'
             'Старт второго этапа российско-европейской программы «ЭкзоМарс» '
             'намечен на период с 26 июля по 11 августа 2020 года. '
             'Ракета-носитель «Протон» выведет в космос перелетный модуль, '
             'который доставит к Марс

В качестве параметров функции передается документ (в случае Python - словарь) с полями и их значениями. 

Если значение должно отвечать не равенству, а другому условию, передается составное значение в виде словаря, ключем у которого будет обозначение оператора, а значением - значение с которым будет проводиться сравнение. Таких пар можно передать несколько.

In [22]:
dictionary.find()[100]

{'_id': ObjectId('637f6f0d893f7efe90694b10'),
 'token': 'исследованием',
 'iniForm': 'исследование',
 'POS': 'NOUN',
 'freq': 1,
 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}

In [26]:
# (freq _какое_ 2) _и_ (token _какое_ 'может')
#for l in dictionary.find({"freq": 2, 'token': 'может'}):
# token _какое_ 'может'
# for l in dictionary.find({'token': 'ученые'}):
for l in dictionary.find({'token': 'ученые', 'freq': 16}):
    print(l)


{'_id': ObjectId('637f6f0e893f7efe90694c6b'), 'token': 'ученые', 'iniForm': 'учёный', 'POS': 'NOUN', 'freq': 16, 'docs': [ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f12893f7efe906956ff'), ObjectId('637f6f14893f7efe90695b7e'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c')]}


In [24]:
# freq _какое_ меньше _чего_ 5
for l in lemmas.find({"freq":{"$lt": 5}})[:5]:
    print(l)

print("----")
# freq _какое_ меньше _чего_ 5
for l in dictionary.find({"freq":{"$lt": 5}})[:5]:
    print(l)
    
print("----")
# freq _какое_ ((больше _чего_ 5) _и_ (меньше _чего_ 9))
for l in dictionary.find({"freq":{"$gt": 5, "$lt": 9}})[:5]:
    print(l)


{'_id': ObjectId('637f6f0d893f7efe906949dd'), 'iniForm': 'розалинда', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949e0'), 'iniForm': 'франклин', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949e9'), 'iniForm': 'отправиться', 'POS': 'INFN', 'freq': 1, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949ef'), 'iniForm': 'красный', 'POS': 'ADJF', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949f8'), 'iniForm': 'лето', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f21893f7efe906969af')]}
----
{'_id': ObjectId('637f6f0d893f7efe906949db'), 'token': 'Марсоход', 'iniForm': 'марсоход', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949de'), 'token': 'Розалинд', 'iniForm': '

In [28]:
text_collection.find({"_id":ObjectId('637f6f0e893f7efe90694c28')})[0]

{'_id': ObjectId('637f6f0e893f7efe90694c28'),
 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators',
 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ',
 'art_text': 'Физики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До си

Те же запросы можно представить в виде вот таких деревьев.

```
{"freq":{"$ge": 5}}  
  
freq  
  |  
  ge  
  |  
  5  
  
   ge  
  /  \  
freq  5  
  
{"freq":{"$gt": 5, "$lt": 9}}  
  
freq  
   |  
   &  
  /  \  
 gt  lt  
  |   |  
  5   9  
    
{"freq":{"$gt":5,"$lt":9}}  
```

In [29]:
# Можно записать и так.
# (freq _какое_ (больше _чего_ 5)() _и_ (freq _какое_ (меньше _чего_ 9))
for l in dictionary.find({"freq":{"$gt": 5}, "freq": {"$lt": 9}}):
    print(l)

{'_id': ObjectId('637f6f0d893f7efe906949db'), 'token': 'Марсоход', 'iniForm': 'марсоход', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949de'), 'token': 'Розалинд', 'iniForm': 'розалинда', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949e1'), 'token': 'Франклин', 'iniForm': 'франклин', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949e7'), 'token': 'должен', 'iniForm': 'должный', 'POS': 'ADJS', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f12893f7efe906956ff')]}
{'_id': ObjectId('637f6f0d893f7efe906949ea'), 'token': 'отправиться', 'iniForm': 'отправиться', 'POS': 'INFN', 'freq': 1, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949f0'), 'token': 'Красной', 'iniForm': 'красный', 'POS': 'ADJF', 'freq': 3, 'docs': [ObjectId('637f6

In [30]:
res1 = lemmas.find({"freq":{"$gt": 5}})
res1[0]['iniForm'], type(res1[0])

('который', dict)

Попробуем найти все документы со сложностью равной 2.3 (статья про ЭкзоМарс).

In [33]:
for l in text_collection.find({'art_info': {'difficulty': '4.2'}}):
    print(l)

In [38]:
text_collection.find({'art_info': {'art_date': '20.01.20', 
                                   'art_time': '10:40', 
                                   'difficulty': '4.2', 
                                   'author': '\n                                      \n                  \n                  Олег Макаров\n                '
                                  }
                     }
                    )[0]

{'_id': ObjectId('637f6f0e893f7efe90694c28'),
 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators',
 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ',
 'art_text': 'Физики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До си

Ничего не найдено, так как такая запись предполагает полное совпадение, то есть поиск поддокумента с единственным полем и этим значением. Если полей будет несколько, роль будет играть даже порядок следования полей в базе!

То, что мы хотели сделать,нужно сделать вот так.

In [34]:
for l in text_collection.find({'art_info.difficulty': '4.2'}):
    print(l)

{'_id': ObjectId('637f6f0e893f7efe90694c28'), 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators', 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ', 'art_text': 'Физики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До сих п

Можно выбирать не все поля, а только те, которые нам необходимы.

In [39]:
for t in text_collection.find(projection=['art_info.difficulty','text_name']):
    print(t)

{'_id': ObjectId('637f6f0d893f7efe906949d9'), 'text_name': '\n            Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания\n          ', 'art_info': {'difficulty': '2.3'}}
{'_id': ObjectId('637f6f0e893f7efe90694c28'), 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ', 'art_info': {'difficulty': '4.2'}}
{'_id': ObjectId('637f6f0f893f7efe90694ff3'), 'text_name': '\n            Пилотируемый полет Crew Dragon к МКС состоится в первой половине 2020 года\n          ', 'art_info': {'difficulty': '1.7'}}
{'_id': ObjectId('637f6f10893f7efe90695273'), 'text_name': '\n            Сон помог вознаграждению улучшить зрительное обучение\n          ', 'art_info': {'difficulty': '3.9'}}
{'_id': ObjectId('637f6f12893f7efe906956ff'), 'text_name': '\n            Машинное обучение помогло распознать человека по танцу\n          ', 'art_info': {'difficulty': '1.9'}}
{'_id': ObjectId('637f6f13893f7efe90695970'), 'text_name': '\n            А

Для поиска в массивах одного значения можно просто проводить поиск, БД сама будет искать объекты в массиве.

Если мы ищем несколько значений в массиве, необходимо использовать соответствующие предикаты: `$in` для поиска вхождения одного из элементов, `$all` - поиска всех элементов, и т.д.

Обратите внимание, для поиска мы используем не строку, а объект `ObjectId`, импортированный из библиотеки `bson`.

Помимо этого, во втором цикле в поле projection мы передаем не только какие поля надо выводить, но и какие поля выводить не надо. Так, поле \_id, хранящее идентификатор документа, по умолчанию должно выводиться. В примере ниже оно подавляется при помощи `"_id":False`.

In [40]:
for l in dictionary.find({'token': 'ученые'}):
    print(l)
    print(l['docs'][0])
    for d in dictionary.find({"docs": {"$in":[l['docs'][0]]}})[:5]:
        print(d)
    

{'_id': ObjectId('637f6f0e893f7efe90694c6b'), 'token': 'ученые', 'iniForm': 'учёный', 'POS': 'NOUN', 'freq': 16, 'docs': [ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f12893f7efe906956ff'), ObjectId('637f6f14893f7efe90695b7e'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c')]}
637f6f0e893f7efe90694c28
{'_id': ObjectId('637f6f0d893f7efe906949ed'), 'token': 'к', 'iniForm': 'к', 'POS': 'PREP', 'freq': 25, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f13893f7efe90695970'), ObjectId('637f6f18893f7efe9069601c'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c'), ObjectId('637f6f21893f7efe906969af')]}
{'_id': ObjectId('637f6f0d893f7efe90694a14'), 'token': 'среды', 'iniForm': 'среда', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('63

In [42]:
for d in dictionary.find({"docs": ObjectId('637f6f0e893f7efe90694c28')})[:5]:
    print(d)
    
print("-----")
for d in dictionary.find({"docs": {"$in": [ObjectId('637f6f0e893f7efe90694c28')]}})[:5]:
    print(d)
    
print("-----")
for d in dictionary.find({"docs": {"$all": [ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f10893f7efe90695273')]}},
                         projection={"_id":False, "token":True, "docs":True}):
    print(d)    

{'_id': ObjectId('637f6f0d893f7efe906949ed'), 'token': 'к', 'iniForm': 'к', 'POS': 'PREP', 'freq': 25, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f13893f7efe90695970'), ObjectId('637f6f18893f7efe9069601c'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c'), ObjectId('637f6f21893f7efe906969af')]}
{'_id': ObjectId('637f6f0d893f7efe90694a14'), 'token': 'среды', 'iniForm': 'среда', 'POS': 'NOUN', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28')]}
{'_id': ObjectId('637f6f0d893f7efe90694a17'), 'token': 'в', 'iniForm': 'в', 'POS': 'PREP', 'freq': 189, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f12893f7efe906956ff'), ObjectId('637f6f13893f7efe90695970'), ObjectId

In [43]:
# А это мы просто смотрим сколько текстов добавили в базу.
# В качестве параметра передается условие, считаются только удовлетворяющие ему документы.
# В данном случае нам нужны все.
print(text_collection.count_documents({}))

# Ищем количество слов с частотой встречаемости от 6 до 9.
print(lemmas.count_documents({"freq":{"$gt": 5, "$lt": 10}}))

# Здесь ищем слова с частотой встречаемости от 6 до 9, 
# просим не выводить идентификатор и отсортировать всё по частоте.
for l in lemmas.find({"freq":{"$gt": 5, "$lt": 10}}, {"_id":False}).sort("iniForm")[:5]:
    print(l)
print("----")
# Здесь ищем слова с частотой встречаемости от 6 до 9, 
# просим вывести токен и частоту, но не выводить идентификатор и отсортировать всё по частоте.
for l in dictionary.find({"freq":{"$gt": 5}}, {"token": True, "freq":True, "_id":False}, 
                         skip=25, limit=5).sort("freq"):
    print(l)


13
79
{'iniForm': 'алгоритм', 'POS': 'NOUN', 'freq': 7, 'docs': [ObjectId('637f6f12893f7efe906956ff'), ObjectId('637f6f16893f7efe90695e45'), ObjectId('637f6f18893f7efe9069601c')]}
{'iniForm': 'аппарат', 'POS': 'NOUN', 'freq': 6, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f13893f7efe90695970')]}
{'iniForm': 'атмосфера', 'POS': 'NOUN', 'freq': 8, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f13893f7efe90695970'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c')]}
{'iniForm': 'больший', 'POS': 'ADJF', 'freq': 8, 'docs': [ObjectId('637f6f14893f7efe90695b7e'), ObjectId('637f6f16893f7efe90695e45'), ObjectId('637f6f18893f7efe9069601c'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c'), ObjectId('637f6f21893f7efe906969af')]}
{'iniForm': 'влиять', 'POS': 'VERB', 'freq': 6, 'docs': [ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f1d893f7efe9069660c')]}
----
{'token'

Теперь напишем функцию для того, чтобы класть статью в БД. используем для этого функции:
- insert_one (insert, insert_many) - добавляет запись (несколько записей) в выбранную коллекцию.
- find_one_and_update - найти и обновить (аналог update).

В качестве параметра во все функции добавления передается документ (словарь), ключи которого содержат названия полей, а значения - значения этих полей. Значение поля само может являться документом. При поиске часть полей может принимать особые значения, например, `$ne` - не равно, `$size` - смотрим на размер массива, `$inc` - увеличить значение поля на заданную величину и т.д. Более подробно с соответствующими ключами можно ознакомиться <a href="https://metanit.com/nosql/mongodb/" >здесь</a>.

In [44]:
for d in dictionary.find({"docs": {"$size": 1}})[:5]:
    print(d)


{'_id': ObjectId('637f6f0d893f7efe906949db'), 'token': 'Марсоход', 'iniForm': 'марсоход', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949de'), 'token': 'Розалинд', 'iniForm': 'розалинда', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949e1'), 'token': 'Франклин', 'iniForm': 'франклин', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949ea'), 'token': 'отправиться', 'iniForm': 'отправиться', 'POS': 'INFN', 'freq': 1, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe906949f0'), 'token': 'Красной', 'iniForm': 'красный', 'POS': 'ADJF', 'freq': 3, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}


In [12]:
# Создаем морфоанализатор.
morph = pymorphy2.MorphAnalyzer()

In [13]:
def putNPlus1ArticleInMongo(art_url):
    # Загружаем текст статьи и другие ее части.
    art = getArticleTextNPlus1(art_url) 

    # Добавляем запись с информацией о статье.
    # inserted_id позволяет сразу получить идентификатор записи, чтобы потом ссылаться на него там, где это необходимо.
    a_text = {"text_url": art_url, "text_name": art.head, "art_text": art.text, 
              "art_info": {"art_date": art.date, "art_time": art.time, 
                           "difficulty": art.diff, "author": art.author}}
    text_id = text_collection.insert_one(a_text).inserted_id
    print(text_id)

    # Выделяем предложения (просто по точке с пробелом или в конце строки!!!).
    sents = re.split("\.\s|\.$", art.text)
    sent_num = 1
    # Загружаем предложения в базу.
    for s in tqdm(sents):
        # Выделяем слова (просто как группы русских букв!!!).
        words = re.findall("([А-Яа-я]+(\-[А-Яа-я]+)?)", s)
        posit = 1
        # Загрузка слов из предложений.
        for w in words:
            wf = morph.parse(w[0])
            # Провели морфологический анализ и теперь добавляем документ с двумя полями: нач. форма и часть речи.
            # Сперва проверяем есть ли там уже такая запись.
            # Смотрим сколько таких слов нашлось. Если ни одного - надо добавлять.
            if lemmas.count_documents({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                lemma_id = lemmas.insert_one({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS, 
                                              "freq": 1, "docs":[text_id]}).inserted_id
            # А если такое слово уже было, то обновляем частоту встречаемости и в каком документе оно встретиось.
            else:
                inis = lemmas.find({"iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                lemma_id = inis[0]["_id"]
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$inc": {"freq":1}})
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$addToSet": {"docs": text_id}}) 
                lemmas.find_one_and_update({"_id": lemma_id}, 
                                           {"$inc": {"freq": 1}, 
                                            "$addToSet": {"docs": text_id}})
                # Вот таким образом можно добавлять номер документа столько раз, сколько в нем встретилось слово.
                #lemmas.find_one_and_update({"_id": lemma_id}, {"$push": {"docs": text_id}}) 

            # Повторяем операцию для токенов.
            if dictionary.count_documents({"token":w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS}) == 0:
                wf_id = dictionary.insert_one({"token":w[0], 
                                               "iniForm": wf[0].normal_form, 
                                               "POS": wf[0].tag.POS, 
                                               "freq": 1, 
                                               "docs": [text_id]}).inserted_id
            else:
                wrdf = dictionary.find({"token": w[0], "iniForm": wf[0].normal_form, "POS": wf[0].tag.POS})
                wf_id=wrdf[0]["_id"]
                dictionary.find_one_and_update({"_id": wf_id}, {"$inc":  {"freq": 1}}) 
                dictionary.find_one_and_update({"_id": wf_id}, {"$addToSet": {"docs": text_id}}) 
  
            # Добавляем номер предложения, идентификатор словоформы из словаря, позицию слова, из какого оно текста.
            dbsents.insert_one({"sent_id": sent_num, "wordFormId":wf_id, "position": posit, "textId": text_id})
            posit += 1  
        sent_num += 1

In [14]:
# Добавляем статью  в базу.
putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/subwavelength-resonators")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/20/crew-dragon-first-crewed-flight")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/18/visual-perception")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/01/18/dancing-recognition")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/06/27/juno-pictures")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/04/12/alignment-of-jets")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2016/06/29/Ultra-Deep-Survey")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2018/03/20/balance")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2018/03/20/abel-prize-2018")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2020/10/12/srb-electrode")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2021/02/19/friction-on-ice")
putNPlus1ArticleInMongo("https://nplus1.ru/news/2021/04/07/illex-coindetii")

637f6f0d893f7efe906949d9


100%|███████████████████████████████████████████| 13/13 [00:00<00:00, 26.55it/s]


637f6f0e893f7efe90694c28


100%|███████████████████████████████████████████| 29/29 [00:00<00:00, 31.16it/s]


637f6f0f893f7efe90694ff3


...

100%|███████████████████████████████████████████| 31/31 [00:02<00:00, 11.67it/s]


637f6f1d893f7efe9069660c


100%|███████████████████████████████████████████| 34/34 [00:02<00:00, 11.35it/s]


637f6f21893f7efe906969af


100%|███████████████████████████████████████████| 34/34 [00:03<00:00, 11.13it/s]


In [48]:
# Вот так можно удалить всё в базе. Если написать условие, то не всё. 
# Экспериментировать не будем, да?
text_collection.delete_many({})
dictionary.delete_many({})
lemmas.delete_many({})
dbsents.delete_many({})
#for l in dbsents.find():
# print(l)


In [45]:
# Так можно посмотреть все записи, у которых в токене записан список ровно из пяти элементов.
#for l in dictionary.find({"docs":{"$size":5}})[:5]:
for l in dictionary.find({"docs":{"$size":4}})[:5]:
    print(l)
print("=====")
# Так можно посмотреть все записи, у которых есть поле docs.
for l in lemmas.find({"docs":{"$exists":True}})[:5]:
    print(l)

{'_id': ObjectId('637f6f0d893f7efe90694a93'), 'token': 'поверхность', 'iniForm': 'поверхность', 'POS': 'NOUN', 'freq': 7, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f1a893f7efe90696256'), ObjectId('637f6f1d893f7efe9069660c')]}
{'_id': ObjectId('637f6f0d893f7efe90694aa3'), 'token': 'качестве', 'iniForm': 'качество', 'POS': 'NOUN', 'freq': 6, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f14893f7efe90695b7e'), ObjectId('637f6f1a893f7efe90696256')]}
{'_id': ObjectId('637f6f0d893f7efe90694aa9'), 'token': 'работы', 'iniForm': 'работа', 'POS': 'NOUN', 'freq': 5, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f15893f7efe90695d2a'), ObjectId('637f6f18893f7efe9069601c')]}
{'_id': ObjectId('637f6f0d893f7efe90694b35'), 'token': 'чего', 'iniForm': 'что', 'POS': 'NPRO', 'freq': 5, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('

При помощи ключевого слова `$regex' можно искать в базе строки, отвечающие регулярным выражениям.

In [46]:
# iniForm _какое_ описывается_регуляркой _какой_ ^и.+
for d in lemmas.find({'iniForm':{'$regex':'^и.+'}})[:5]:
    print(d)
print("====")    
for d in dictionary.find({'token':{'$regex':'^[А-Я].+'}})[:5]:
    print(d)

{'_id': ObjectId('637f6f0d893f7efe90694a07'), 'iniForm': 'испытание', 'POS': 'NOUN', 'freq': 7, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0f893f7efe90694ff3'), ObjectId('637f6f16893f7efe90695e45')]}
{'_id': ObjectId('637f6f0d893f7efe90694a0a'), 'iniForm': 'имитировать', 'POS': 'PRTF', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9')]}
{'_id': ObjectId('637f6f0d893f7efe90694a49'), 'iniForm': 'июль', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f13893f7efe90695970')]}
{'_id': ObjectId('637f6f0d893f7efe90694b01'), 'iniForm': 'изучение', 'POS': 'NOUN', 'freq': 4, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f13893f7efe90695970'), ObjectId('637f6f14893f7efe90695b7e')]}
{'_id': ObjectId('637f6f0d893f7efe90694b0f'), 'iniForm': 'исследование', 'POS': 'NOUN', 'freq': 12, 'docs': [ObjectId('637f6f0d893f7efe906949d9'), ObjectId('637f6f12893f7efe906956ff'), ObjectId('637f6

Ключевое слово `$or` и другие операторы позволяют применять логические свзки помимо логического И.

In [47]:
# _или_по_списку_условий_ [token _какой_ описывается_регуляркой _какой_ ^И.+, 
#                          token _какой_ описывается_регуляркой _какой_ .+ъ.+]
for d in dictionary.find({'$and':[{'token':{'$regex':'^о.+'}}, {'token':{'$regex':'.+ъ.+'}}, {'freq':2}]})[:5]:
    print(d)

{'_id': ObjectId('637f6f0e893f7efe90694fd2'), 'token': 'объектов', 'iniForm': 'объект', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f0e893f7efe90694c28'), ObjectId('637f6f21893f7efe906969af')]}
{'_id': ObjectId('637f6f1e893f7efe90696648'), 'token': 'объяснить', 'iniForm': 'объяснить', 'POS': 'INFN', 'freq': 2, 'docs': [ObjectId('637f6f1d893f7efe9069660c')]}


Обратите внимание, что при обновлении заменяетя весь документ целиком. Чтобы избежать этого следует использовать ключевое слово `$set`, которое показывает, что обновляется значение только одного поля.

In [48]:
# Просто посмотрим на документы
for d in dictionary.find({'token':{'$regex':'^И.+'}}):
    print(d)

{'_id': ObjectId('637f6f0e893f7efe90694dee'), 'token': 'ИТМО', 'iniForm': 'итмый', 'POS': 'ADJS', 'freq': 1, 'docs': [ObjectId('637f6f0e893f7efe90694c28')]}
{'_id': ObjectId('637f6f19893f7efe90696179'), 'token': 'Ив', 'iniForm': 'ива', 'POS': 'NOUN', 'freq': 1, 'docs': [ObjectId('637f6f18893f7efe9069601c')]}
{'_id': ObjectId('637f6f11893f7efe906956fd'), 'token': 'Ивтушок', 'iniForm': 'ивтушок', 'POS': 'NOUN', 'freq': 2, 'docs': [ObjectId('637f6f10893f7efe90695273'), ObjectId('637f6f12893f7efe906956ff')]}
{'_id': ObjectId('637f6f0e893f7efe90694e2b'), 'token': 'Из', 'iniForm': 'из', 'POS': 'PREP', 'freq': 1, 'docs': [ObjectId('637f6f0e893f7efe90694c28')]}
{'_id': ObjectId('637f6f1b893f7efe9069634d'), 'token': 'Известно', 'iniForm': 'известно', 'POS': 'PRED', 'freq': 1, 'docs': [ObjectId('637f6f1a893f7efe90696256')]}
{'_id': ObjectId('637f6f0f893f7efe906951b9'), 'token': 'Изначально', 'iniForm': 'изначально', 'POS': 'ADVB', 'freq': 1, 'docs': [ObjectId('637f6f0f893f7efe90694ff3')]}
{'_id'

In [49]:
# А теперь найдем первое попавшееся слово, начинающееся с заглавное И и добавим ему новое поле found
dictionary.find_one_and_update({'token':{'$regex':'^И.+'}, 'found':'false'}, {'$set': {'found':'true'}})

In [50]:
for r in dictionary.find({'token': 'Изначально'}):
    print(r)

{'_id': ObjectId('637f6f0f893f7efe906951b9'), 'token': 'Изначально', 'iniForm': 'изначально', 'POS': 'ADVB', 'freq': 1, 'docs': [ObjectId('637f6f0f893f7efe90694ff3')]}


Ничего не получилось, так как поле found отсутствует у всех записей. Попробуем теперь найти первую запись, у которой нет такого поля: 'found':{"$exists":False}} .

In [51]:
dictionary.find_one_and_update({'token':{'$regex':'^А.+'}, 'found':{"$exists":False}}, {'$set': {'found':'true'}})

{'_id': ObjectId('637f6f19893f7efe906961f6'),
 'token': 'Абелевская',
 'iniForm': 'абелёвский',
 'POS': 'ADJF',
 'freq': 1,
 'docs': [ObjectId('637f6f18893f7efe9069601c')]}

In [52]:
# Попробуем найти все документы с полем found.
for d in dictionary.find({'token':{'$regex':'^А.+'}, 'found':{"$exists":True}}):
    print(d)

{'_id': ObjectId('637f6f19893f7efe906961f6'), 'token': 'Абелевская', 'iniForm': 'абелёвский', 'POS': 'ADJF', 'freq': 1, 'docs': [ObjectId('637f6f18893f7efe9069601c')], 'found': 'true'}


#### Важное замечание

Вся структура нашей базы данных отдает реляционностью. Никто не мешал нам сделать документ, вложенный в словарь начальных форм и хранить в нем все формы данного слова. Тогда если что, не надо будет делать запросы, в которых надо делать пересечение двух коллекций.

{"iniForm":"абелёвский", 'POS': 'ADJF', "forms":[{'token': 'Абелевской', "tags":"род ед муж"}, {'token': 'Абелевский', "tags":"им ед муж"}]}

## Работа с географическими данными

MongoDB позволяет работать с географическими данными. Основой является [GEO JSON](https://docs.mongodb.com/manual/reference/geojson/), позволяющий обрабатывать разные виды геолокаций.


In [54]:
text_collection.find_one_and_update(
    {'_id': ObjectId('637f6f0e893f7efe90694c28')},
    {"$set":{'coord':{ 'type': "Point", 'coordinates': [ 20, 15 ] }}})


{'_id': ObjectId('637f6f0e893f7efe90694c28'),
 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators',
 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ',
 'art_text': 'Физики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До си

In [55]:
for a in text_collection.find():
    print(a)

{'_id': ObjectId('637f6f0d893f7efe906949d9'), 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes', 'text_name': '\n            Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания\n          ', 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.\nСтарт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей марсох

In [56]:
for d in text_collection.find({'coord':{'$geoIntersects':{"$geometry": 
{"type": "Polygon",
 "coordinates": [[[15,10], [15,20],[25,20], [25,10], [15,10]]]
}}}}):
    print(d)

{'_id': ObjectId('637f6f0e893f7efe90694c28'), 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators', 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ', 'art_text': 'Физики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До сих п

#### Важное замечание

!!! Если не построить этот индекс, то искаться не будет!!!

In [57]:
#lemmas.create_index([('iniForm', pymongo.ASCENDING), ('POS', pymongo.ASCENDING)], unique=True)

text_collection.create_index([("coord", pymongo.GEOSPHERE)], unique=False)

'coord_2dsphere'

In [58]:
req=text_collection.find(
{
   "coord": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ 19.999 , 15.001 ]
       },
       "$maxDistance": 1000,
       "$minDistance": 0
     }
   }
}
)
#req
for d in req:    
    print(d)

{'_id': ObjectId('637f6f0e893f7efe90694c28'), 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators', 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ', 'art_text': 'Физики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До сих п

In [59]:
for d in text_collection.find({'coord':{'$geoWithin':{"$box": 
[[15,10], [25,20]]
}}}):
    print(d)

{'_id': ObjectId('637f6f0e893f7efe90694c28'), 'text_url': 'https://nplus1.ru/news/2020/01/20/subwavelength-resonators', 'text_name': '\n            Физики заперли свет в нанорезонаторе на рекордно долгое время\n          ', 'art_text': 'Физики создали резонатор размером в несколько сотен нанометров, способный удерживать свет внутри себя на\xa0время, за\xa0которое световая волна совершает более 200 периодов колебаний. На\xa0основе него ученые создали устройство, которое увеличивает частоту входного света в\xa0два раза, а в будущем такие нанорезонаторы могут стать основой для создания оптических средств связи, приборов ночного видения и\xa0компактных сенсоров. Работа опубликована в\xa0журнале Science. \nС\xa0помощью электрооптики можно передавать информацию на\xa0расстояние, считывать и\xa0записывать данные. Для контроля света его нужно уметь удерживать в\xa0малой области пространства на\xa0достаточно долгое время. Но\xa0чем меньше резонатор, тем сложнее удержать в\xa0нем волну. До сих п

In [60]:
#text_collection.create_index([('text_name', pymongo.ASCENDING)], unique=False)
text_collection.create_index([('art_text', pymongo.TEXT)], unique=False)


'art_text_text'

In [61]:
#for d in text_collection.find({'text_name': 'Бактерии помогли получить катализатор для электролиза воды'}):
# for d in text_collection.find({
#   "$text":{"$search": "воды"}}):
        
#for d in text_collection.find({'art_text':{'$regex':'воды'}}):
for d in text_collection.find({ "$text": { "$search": "Американское учеными" } } ):
# for d in text_collection.find({ "$text": { "$search": "Американ" } } ):
    print(d)

{'_id': ObjectId('637f6f14893f7efe90695b7e'), 'text_url': 'https://nplus1.ru/news/2016/04/12/alignment-of-jets', 'text_name': '\n            Астрономы внезапно нашли сонаправленные джеты\n          ', 'art_text': 'Астрономы из\xa0Кейптаунского Университета и\xa0Университета Западно-Капской провинции в\xa0Южной Африке обнаружили массивные черные дыры, джеты которых ориентированы в\xa0одном направлении. Подобное явление на больших масштабах было замечено учеными впервые и не было \nпредсказано существующими теориями, поэтому в будущем его только \nпредстоит изучить. Работа авторов опубликована в\xa0журнале Monthly Notices of\xa0the Royal Astronomical Society, также она доступна на\xa0сайте ArXiv.org.\nИсследователи вели наблюдения в течение трех лет в\xa0радиоволновом диапазоне при помощи телескопа Giant Metrewave Radio Telescope (GMRT) за\xa0участком неба\n. Первоначальной задачей глубокого обзора было изучение самых слабых радиоисточников во\xa0Вселенной. Упорядоченно ориентированные д

In [62]:
for d in text_collection.find():
    print(d)

{'_id': ObjectId('637f6f0d893f7efe906949d9'), 'text_url': 'https://nplus1.ru/news/2020/01/20/Exomars-2020-rover-yes', 'text_name': '\n            Марсоход миссии «ЭкзоМарс-2020» прошел термовакуумные испытания\n          ', 'art_text': 'Марсоход «Розалинд Франклин», который должен отправиться к Красной планете этим летом, успешно прошел заключительные термовакуумные испытания, имитировавшие\xa0условия марсианской среды, в которых предстоит работать роверу,\xa0сообщается на сайте ESA.\nСтарт второго этапа российско-европейской программы «ЭкзоМарс» намечен на период с 26 июля по 11 августа 2020 года. Ракета-носитель «Протон» выведет в космос перелетный модуль, который доставит к Марсу десантный модуль, содержащий в себе автономную научную станцию «Казачок»\xa0и марсоход «Розалинд Франклин». Посадка на поверхность Красной планеты должна состояться 19 марта 2021 года, в качестве места работы аппаратов выбрана\xa0равнина Оксия в северном полушарии Марса, где есть сухие русла. Задачей марсох